In [115]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [116]:
data = pd.read_csv(r'..\..\Census Data\American Community Survey\Housing Costs B25070\5 Year Estimates\ACSDT5Y2022.B25070-Data.csv')
geodata_CA = gpd.read_file(r'..\..\Census Data\GeoData Tiger Line\tl_2023_06_tract.shp')
demographic_data_raw = pd.read_csv(r'..\..\Census Data\American Community Survey\Demographics DP05\ACSDP5Y2022.DP05-Data.csv', low_memory=False)

In [117]:
# Simplifying the geometry a bit to reduce file size for sharing
#geodata_CA['geometry'] = geodata_CA.geometry.simplify(tolerance=0.001)

In [118]:
# limiting data to include only LA County and Orange County (based on Country FIPS codes 037 and 059, respectively)
geodata_CA = geodata_CA.loc[(geodata_CA['COUNTYFP'].str.contains('037'))|
                            (geodata_CA['COUNTYFP'].str.contains('059'))]

In [ ]:
# keeping the percentage-based demographic columns (getting rid of margin of error data, total counts, unnamed: 366)
drop_these = []
for column in demographic_data_raw.columns:
    if (not column.endswith('PE')) and (column not in ['GEO_ID', 'NAME']) or (column.endswith('M')):
        drop_these.append(column)
print(drop_these)

In [120]:
demographic_data = demographic_data_raw.drop(columns=drop_these, axis=1)

In [121]:
# dropping margin of error columns
data = data.drop(columns= ['B25070_001M', 'B25070_002M', 'B25070_003M', 'B25070_004M', 'B25070_005M', 'B25070_006M', 'B25070_007M', 'B25070_008M', 'B25070_009M', 'B25070_010M', 'B25070_011M', 'Unnamed: 24'], axis = 1)

In [122]:
# Changing headers
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data = data.reset_index()

demographic_data.columns = demographic_data.iloc[0,:]
demographic_data = demographic_data.iloc[1:,:]
demographic_data = demographic_data.reset_index()

In [123]:
# There are 4 columns with duplicated column names in this dataset. This removes them
demographic_data = demographic_data.loc[:, ~demographic_data.columns.duplicated()]

In [124]:
def parse_GEOID(long_GEOID):
    _, short_form = long_GEOID.split('US')
    return short_form

In [125]:
# Match GEOIDs from both dataframes by parsing the longer version from the ACS data
data['GEOID'] = data['Geography'].apply(parse_GEOID)
demographic_data['GEOID'] = demographic_data['Geography'].apply(parse_GEOID)

In [126]:
# drop now-defunct Geography & old index columns
data= data.drop(columns = ['Geography', 'index'], axis =1)
demographic_data= demographic_data.drop(columns = ['Geography', 'index'], axis =1)

In [127]:
# "Burdened" here means spending more than 30% of the household income on rent
def calculate_burdened_individuals (row):
    over_30 = (row[7:11].astype('float')).sum()
    percentage_over_30 = over_30 / float(row.iloc[1]) if float(row.iloc[1]) != 0 else 0
    return over_30, percentage_over_30

In [128]:
result = pd.DataFrame()
result['number_burdened'], result['proportion_burdened'] = zip(*data.apply(calculate_burdened_individuals, axis = 1))

In [129]:
# Add the calculated burdened data to the original dataframe
data = data.join(result)

In [ ]:
# looking at census tracts with highest proportions of cost-burdened renters
data['Geographic Area Name'].loc[data['proportion_burdened'] >= (data['proportion_burdened'].quantile(0.95))]

In [ ]:
# confirming GEOIDs match
geodata_CA['GEOID']

In [ ]:
data['GEOID']

In [ ]:
demographic_data['GEOID']

In [134]:
# Convert GeoIDs to strings
geodata_CA['GEOID'] = geodata_CA['GEOID'].astype(str)
data['GEOID'] = data['GEOID'].astype(str)
demographic_data['GEOID'] = demographic_data['GEOID'].astype(str)

In [ ]:
# Check the unique GEOID values in each dataset
gdf_geoids = set(geodata_CA['GEOID'].unique())
renter_geoids = set(data['GEOID'].unique())
demo_geoids = set(demographic_data['GEOID'].unique())

# Find the GEOIDs present in gdf but not in renter data & visa versa
missing_in_renter = gdf_geoids - renter_geoids
missing_in_gdf = renter_geoids - gdf_geoids
missing_in_demo = gdf_geoids - demo_geoids

print(f"GEOIDs in shapefile but not in renter data: {len(missing_in_renter)}")
print(f"GEOIDs in renter data but not in shapefile: {len(missing_in_gdf)}")
print(f"GEOIDs in demographic data but not in shapefile: {len(missing_in_demo)}")


In [136]:
# Merge geodata + renter dataframes
geodata_merged = geodata_CA.merge(data, how= 'inner', on='GEOID')
geodata_merged = geodata_merged.merge(demographic_data, how= 'inner', on= ['GEOID', 'Geographic Area Name'])

In [137]:
# Dropping columns that aren't necessary
geodata_merged = geodata_merged.drop(columns=['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOIDFQ', 'NAME', 'MTFCC', 'FUNCSTAT', 'ALAND','AWATER', 'INTPTLAT', 'INTPTLON','NAMELSAD'], axis = 1)

In [138]:
# Giving columns reader-friendly names for map legend
geodata_merged = geodata_merged.rename(columns={
    "Estimate!!Total:": "Total Households", "Estimate!!Total:!!Less than 10.0 percent": "<10% Income Spent on Rent", 'Estimate!!Total:!!10.0 to 14.9 percent': '10-14.9% Income Spent on Rent', 'Estimate!!Total:!!15.0 to 19.9 percent': '15-19.9% Income Spent on Rent', 'Estimate!!Total:!!20.0 to 24.9 percent': '20-24.9% Income Spent on Rent',  'Estimate!!Total:!!25.0 to 29.9 percent': '25-29.9% Income Spent on Rent', 'Estimate!!Total:!!30.0 to 34.9 percent': '30-34.9% Income Spent on Rent', 'Estimate!!Total:!!35.0 to 39.9 percent': '35-39.9% Income Spent on Rent', 'Estimate!!Total:!!40.0 to 49.9 percent': '40-49.9% Income Spent on Rent', 'Estimate!!Total:!!50.0 percent or more': '50+% Income Spent on Rent', 'Estimate!!Total:!!Not computed': 'Rent Expenditure Unknown', 'number_burdened': 'Number of Rent-Burdened Households (>30% Expenditure)', 'proportion_burdened': 'Proportion of Rent Burdened Households', 'Percent!!Race alone or in combination with one or more other races!!Total population!!Black or African American': 'Percent Black (alone or in combination with other races)', 'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)': 'Percent Hispanic or Latino (alone or in combination with other races)', 'Percent!!Race alone or in combination with one or more other races!!Total population!!Asian': 'Percent Asian (alone or in combination with other races)', 'Percent!!RACE!!Total population!!One race!!White': 'Percent White'})

In [ ]:
geodata_CA.plot()

In [ ]:
geodata_CA.plot()

In [ ]:
# Plotting static image
fig, ax = plt.subplots(figsize=(5,5))
geodata_merged.plot(column = 'Proportion of Rent Burdened Households', cmap ='RdYlGn_r', legend = True, ax=ax)
ax.set_title('Proportion of Cost-Burdened Renter Households per Census Tract', fontsize=16)
#plt.savefig('CA Cost Burdened Households.png')

In [142]:
#
geodata_merged['Proportion of Rent Burdened Households'] = pd.to_numeric(
    geodata_merged['Proportion of Rent Burdened Households'],
    errors='coerce')
geodata_merged['Percent Black (alone or in combination with other races)'] = pd.to_numeric(
    geodata_merged['Percent Black (alone or in combination with other races)'],
    errors='coerce'
)

In [ ]:
# Making a non-interactable/static map with layers 
# I don't like how this looks, but maybe this code can be useful for someone else?
fig, ax = plt.subplots(figsize=(15, 10))
geodata_merged.plot(ax=ax, color='lightgray', edgecolor='black')
race_layer = geodata_merged.plot(ax=ax, column='Percent Black (alone or in combination with other races)', scheme='quantiles', legend=True, alpha=0.25)
rent_layer = geodata_merged.plot(ax=ax, column='Proportion of Rent Burdened Households', cmap = 'RdYlGn_r', legend=True, alpha=0.25)
ax.axis('off')
ax.set_title('Static Multi-Layer Map', fontsize=16)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper right')

In [ ]:
# Plotting interactable map w/ layers to allow additional demographics data later
import folium

# Base map has rent burdened choropleth overlay
map = geodata_merged.explore(
    column= 'Proportion of Rent Burdened Households', 
    name= 'Proportion of Rent Burdened Households', 
    cmap='RdYlGn_r', 
    legend= True, 
    figsize= (15,15),
    tooltip= [
        'Geographic Area Name', 'Total Households', r'Number of Rent-Burdened Households (>30% Expenditure)', 'Proportion of Rent Burdened Households', 'Percent White', 'Percent Black (alone or in combination with other races)', 'Percent Hispanic or Latino (alone or in combination with other races)', 'Percent Asian (alone or in combination with other races)'])

geodata_merged.explore(
    column='Percent White', 
    name='Ethnicity Overlay', 
    cmap='Blues_r', 
    legend=False,
    tooltip=[
        'Geographic Area Name', 'Percent White', 'Percent Black (alone or in combination with other races)', 'Percent Hispanic or Latino (alone or in combination with other races)', 'Percent Asian (alone or in combination with other races)'
    ],
    figsize=(15, 15),
    m=map  
)
folium.LayerControl().add_to(map)
map

In [146]:
# save as HTML file for sharing
map.save("LA + Orange Counties.html")